# 02_entiry-extract
- entity extraction from news description using trasnformer `Davlan/xlm-roberta-base-ner-hrl`

### install extra library

In [ ]:
!pip install transformers torch
!pip install sentencepiece

### read stored data from DataPlatform

In [663]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [664]:
dir_raw_id = 288
dir_process_id = 289

In [666]:
datanode = DataNode()

Init DataNode sucessful!


In [667]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='news.json'))
meta

{'owner': 'data_engineer',
 'human_size': '104.50 KB',
 'name': 'news.json',
 'description': 'file news.json',
 'directory': 288,
 'path': 'data_engineer/dsi-314/raw/news.json'}

In [668]:
data = json.load(fp)
data

[{'title': "Allemagne-Uniper pourrait avoir besoin de 40 milliards d'euros supplémentaires-Handelsblatt",
  'link': 'https://www.zonebourse.com/cours/action/UNIPER-SE-31299895/actualite/Allemagne-Uniper-pourrait-avoir-besoin-de-40-milliards-d-euros-supplementaires-Handelsblatt-42046543/?utm_medium=RSS&utm_content=20221020',
  'keywords': ['Actualités des sociétés'],
  'creator': None,
  'video_url': None,
  'description': '(zonebourse.com) Le géant allemand de l\'énergie Uniper pourrait avoir besoin d\'une rallonge de 40 milliards d\'euros , rapporte jeudi le journal Handelsblatt. En septembre, le gouvernement allemand avait décidé de nationaliser Uniper, asphyxié par les coupures de gaz russe, et promis de mettre en place un mécanisme de "frein d\'urgence" pour les...https://www.zonebourse.com/cours/action/UNIPER-SE-31299895/actualite/Allemagne-Uniper-pourrait-avoir-besoin-de-40-milliards-d-euros-supplementaires-Handelsblatt-42046543/?utm_medium=RSS&utm_content=20221020',
  'content':

### NER (NameEntitiesRegcognition)
- extract entity from news description

In [669]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [670]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [10]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [671]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [674]:
text = data[0]['description']
text

'(zonebourse.com) Le géant allemand de l\'énergie Uniper pourrait avoir besoin d\'une rallonge de 40 milliards d\'euros , rapporte jeudi le journal Handelsblatt. En septembre, le gouvernement allemand avait décidé de nationaliser Uniper, asphyxié par les coupures de gaz russe, et promis de mettre en place un mécanisme de "frein d\'urgence" pour les...https://www.zonebourse.com/cours/action/UNIPER-SE-31299895/actualite/Allemagne-Uniper-pourrait-avoir-besoin-de-40-milliards-d-euros-supplementaires-Handelsblatt-42046543/?utm_medium=RSS&utm_content=20221020'

In [675]:
output = nlp(text)
output

[{'entity': 'B-ORG',
  'score': 0.7730833,
  'index': 2,
  'word': 'zone',
  'start': 1,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.9989485,
  'index': 16,
  'word': '▁Uni',
  'start': 47,
  'end': 51},
 {'entity': 'I-ORG',
  'score': 0.990167,
  'index': 17,
  'word': 'per',
  'start': 51,
  'end': 54},
 {'entity': 'B-ORG',
  'score': 0.994011,
  'index': 41,
  'word': '▁Handels',
  'start': 143,
  'end': 151},
 {'entity': 'I-ORG',
  'score': 0.98642,
  'index': 42,
  'word': 'blatt',
  'start': 151,
  'end': 156},
 {'entity': 'B-ORG',
  'score': 0.5415239,
  'index': 48,
  'word': '▁gouvernement',
  'start': 174,
  'end': 187},
 {'entity': 'B-ORG',
  'score': 0.9993575,
  'index': 55,
  'word': '▁Uni',
  'start': 225,
  'end': 229},
 {'entity': 'I-ORG',
  'score': 0.9956554,
  'index': 56,
  'word': 'per',
  'start': 229,
  'end': 232},
 {'entity': 'B-ORG',
  'score': 0.7513807,
  'index': 103,
  'word': 'UNI',
  'start': 388,
  'end': 391},
 {'entity': 'B-LOC',
  'score': 0.8983

In [676]:
post_process(output)

[{'type': 'I-ORG', 'text': 'Uniper'},
 {'type': 'I-ORG', 'text': 'Handelsblatt'},
 {'type': 'B-ORG', 'text': 'gouvernement'},
 {'type': 'I-ORG', 'text': 'Uniper'}]

### Process news to spo

In [681]:
spo = []
for news in tqdm(data):
    description = news.get('description') 
    if description == None:
        continue
    ner = nlp(description)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'subject': news.get('title', ""),
            'predicate': kw.get('type', ""),
            'object': kw.get('text', "")
        })

  0%|          | 0/100 [00:00<?, ?it/s]

In [684]:
df = pd.DataFrame(spo)
df['predicate'] = df['predicate'].apply(lambda x: x[2:])
df

,subject,predicate,object
0,Allemagne-Uniper pourrait avoir besoin de 40 m...,ORG,Uniper
1,Allemagne-Uniper pourrait avoir besoin de 40 m...,ORG,Handelsblatt
2,Allemagne-Uniper pourrait avoir besoin de 40 m...,ORG,gouvernement
3,Allemagne-Uniper pourrait avoir besoin de 40 m...,ORG,Uniper
4,FMC FRESENIUS MEDICAL CARE AG & CO KGAA : Jeff...,PER,JamesVane-Tempest
...,...,...,...
270,Comment des juges non-professionnels peuvent-i...,PER,Goumot
271,Comment des juges non-professionnels peuvent-i...,ORG,BFMBusiness
272,Comment des juges non-professionnels peuvent-i...,ORG,Business
273,Comment des juges non-professionnels peuvent-i...,ORG,BFMBusiness


### write DataNode to DataPlatform

In [685]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='news.json')])

File spo.parquet alrady exists, do you want to replace y/n :  y


{'sucess': True,
 'file_id': 1353,
 'path': 'data_engineer/dsi-314/process/spo.parquet',
 'lineage': True}